In [1]:
"""
By default the below script will download most recent 100 comments of a youtube video
You can change the default filter (line 30 onwards)

Variables :
COMMENT_LIMIT : How many comments you want to download 
SORT_BY_POPULAR : filter comments by popularity (0 for True , 1 for false)
SORT_BY_RECENT : filter comments by recently posted (0 for True , 1 for false)
"""

import pandas as pd                                                      #for working with data frames
import json
import os                                                                ##for file and system operations
import sys                                                               ##for file and system operations
import re
import time                                                             #for time-related operations

import csv
import datetime

import requests                                                             #for making HTTP requests

# pandas dataframe display configuration
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

YOUTUBE_COMMENTS_AJAX_URL = 'https://www.youtube.com/comment_service_ajax'

USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'
# csv file name
FILE_NAME = '廣東話.csv'

# set parameters
# filter comments by popularity or recent, 0:False, 1:True
SORT_BY_POPULAR = 0
# default recent
SORT_BY_RECENT = 0
# set comment limit
COMMENT_LIMIT = 100


###
### The script defines regular expressions (YT_CFG_RE and YT_INITIAL_DATA_RE)
### for extracting YouTube configuration data and initial data from the HTML response.
##
YT_CFG_RE = r'ytcfg\.set\s*\(\s*({.+?})\s*\)\s*;'
YT_INITIAL_DATA_RE = r'(?:window\s*\[\s*["\']ytInitialData["\']\s*\]|ytInitialData)\s*=\s*({.+?})\s*;\s*(?:var\s+meta|</script|\n)'

In [2]:
def regex_search(text, pattern, group=1, default=None):
    match = re.search(pattern, text)
    return match.group(group) if match else default
###
# The regex_search function takes three parameters: text, pattern, and group. 
# It uses regular expressions to search for a pattern in the given text and returns the matched group. 
# If no match is found, it returns the default value.
###




def ajax_request(session, endpoint, ytcfg, retries=5, sleep=20):
    url = 'https://www.youtube.com' + endpoint['commandMetadata']['webCommandMetadata']['apiUrl']
    
    data = {'context': ytcfg['INNERTUBE_CONTEXT'],
            'continuation': endpoint['continuationCommand']['token']}

    for _ in range(retries):
        response = session.post(url, params={'key': ytcfg['INNERTUBE_API_KEY']}, json=data)
        if response.status_code == 200:
            return response.json()
        if response.status_code in [403, 413]:
            return {}
        else:
            time.sleep(sleep)
# The ajax_request function takes several parameters: session, endpoint, ytcfg, retries, and sleep. 
# It constructs the URL for the AJAX request using the endpoint information and the base URL of YouTube.
# It prepares the data payload for the request using the ytcfg information and the continuation token from the endpoint.

# The function then makes a POST request to the constructed URL with the necessary parameters.
# If the response status code is 200, it returns the JSON response. 
# If the response status code is 403 or 413, it returns an empty dictionary.
# Otherwise, it waits for a specified amount of time (defined by sleep) and retries the request a specified number of times (defined by retries).


            
            
            
import requests         # requests for making HTTP requests
from bs4 import BeautifulSoup    # for parsing HTML
import json      # for working with JSON data
import re        # for regular expressions
import time      # for introducing delays        
            

## The function below to download comments

            
def download_comments(YOUTUBE_VIDEO_URL, sort_by=SORT_BY_RECENT, language=None, sleep=0.1):
    session = requests.Session()
    session.headers['User-Agent'] = USER_AGENT     # starts by creating a session object and setting the User-Agent header
    response = session.get(YOUTUBE_VIDEO_URL)

    if 'uxe=' in response.request.url:
        session.cookies.set('CONSENT', 'YES+cb', domain='.youtube.com')
        response = session.get(YOUTUBE_VIDEO_URL)
## It makes a GET request to the YouTube video URL and checks if the request is redirected.
## If it is redirected, it sets a cookie and makes another request to the URL.
        
    html = response.text  # The HTML response from the request is stored in the html variable.
    

    #USE BeautifulSoup to parse the HTML to extract the title of video
    soup = BeautifulSoup(html,'html.parser')
    video_title = soup.title.string.strip()    
        
    
    ytcfg = json.loads(regex_search(html, YT_CFG_RE, default=''))
    if not ytcfg:            # Unable to extract configuration
        return 
    if language:        # updates the language in the ytcfg data.
        ytcfg['INNERTUBE_CONTEXT']['client']['hl'] = language
    ##  extracts the YouTube configuration data (ytcfg) using regular expressions and JSON parsing.
    ##  If the configuration data cannot be extracted, the function returns.
        
    data = json.loads(regex_search(html, YT_INITIAL_DATA_RE, default=''))
    
    #The script extracts the initial data of the video using regular expressions and JSON parsing.
    

    section = next(search_dict(data, 'itemSectionRenderer'), None) 
    #searches for the section containing the comments and retrieves the first continuation item renderer.
    renderer = next(search_dict(section, 'continuationItemRenderer'), None) if section else None
    if not renderer:        
        return             #If there is no continuation item renderer, it indicates that comments are disabled, and the function returns.

    needs_sorting = sort_by != SORT_BY_POPULAR
    # If sorting is required (not sorting by popularity), it searches for the sort menu and selects the desired sorting option. 
    # If the sorting fails, an exception is raised.
    
    continuations = [renderer['continuationEndpoint']]
    while continuations:
        continuation = continuations.pop()
        response = ajax_request(session, continuation, ytcfg)

        if not response:
            break
        if list(search_dict(response, 'externalErrorMessage')):
            raise RuntimeError('Error returned from server: ' + next(search_dict(response, 'externalErrorMessage')))

        if needs_sorting:
            sort_menu = next(search_dict(response, 'sortFilterSubMenuRenderer'), {}).get('subMenuItems', [])
            if sort_by < len(sort_menu):
                continuations = [sort_menu[sort_by]['serviceEndpoint']]
                needs_sorting = False
                continue
            raise RuntimeError('Failed to set sorting')

        actions = list(search_dict(response, 'reloadContinuationItemsCommand')) + \
                  list(search_dict(response, 'appendContinuationItemsAction'))
        for action in actions:
            
            for item in action.get('continuationItems', []):
                if action['targetId'] == 'comments-section':
                    # Process continuations for comments and replies.
                    continuations[:0] = [ep for ep in search_dict(item, 'continuationEndpoint')]
                if action['targetId'].startswith('comment-replies-item') and 'continuationItemRenderer' in item:
                    # Process the 'Show more replies' button
                    continuations.append(next(search_dict(item, 'buttonRenderer'))['command'])
        ###
        # It retrieves the comments and continuation items using the ajax_request function. 
        # It processes the response by iterating over the comments and extracting relevant information 
        # such as the comment ID, text, timestamp, author, channel, votes, photo, and heart status.
        ###
        for comment in reversed(list(search_dict(response, 'commentRenderer'))):  #The reversed function is then used to reverse the order of the list, so the comments are processed in reverse order (from the most recent to the oldest).
               
        ##This line iterates over the list of comment renderers extracted from the YouTube API response.
        ##The search_dict function is used to find all instances of the "commentRenderer" key in the response JSON, and the result is converted to a list.
        

       
             yield {'Video_title':video_title,   ##The yield statement is used to define a generator function. When this statement is encountered, it pauses the function's execution, "yields" a value, and remembers its current state.
                   
                   'Comment_id': comment['commentId'],
                   
                   'Comment_content': ''.join([c['text'] for c in comment['contentText'].get('runs', [])]), 
                    #The .get() method is used to safely access nested properties of the comment object to handle cases where a property might be missing.
                    'Time': comment['publishedTimeText']['runs'][0]['text'],
                   'Username': comment.get('authorText', {}).get('simpleText', ''),
                   'Channel_user': comment['authorEndpoint']['browseEndpoint'].get('browseId', ''),
                   'Likes': comment.get('voteCount', {}).get('simpleText', '0'),
                   'Photo_thumbnails': comment['authorThumbnail']['thumbnails'][-1]['url'],
                   'Hearted_by_video_publisher': next(search_dict(comment, 'isHearted'), False)}
        
        ###
        # The next(search_dict(comment, 'isHearted'), False) expression is used to check if the comment has been hearted (liked) by the viewer.
        # The search_dict function is used to find the "isHearted" key in the comment object, and the next function returns its value. 
        # If the key is not found, the default value is set to False.
        ###
   
       ##The extracted information is yielded as a dictionary for each comment.
    
    # The dictionary is created using a dictionary literal syntax (curly braces {}), and the values are extracted from the comment object, 
    # which represents an individual comment renderer in the loop.
        
        time.sleep(sleep)   #After each iteration, the function introduces a delay specified by the sleep parameter.

        
        
def search_dict(partial, search_key): # partial:the nested dictionary or list to search, search_key: the key to search for
    stack = [partial]    #The stack is used to keep track of the nested levels of the dictionary or list that need to be explored.
    while stack:
        current_item = stack.pop()
        if isinstance(current_item, dict):
            for key, value in current_item.items():
                if key == search_key:
                    yield value
                else:
                    stack.append(value)
        elif isinstance(current_item, list):
            for value in current_item:
                stack.append(value)
###
#  If current_item is a dictionary, the function iterates over its key-value pairs. 
#  For each key-value pair, it checks if the key matches the search_key. If there is a match, it yields the corresponding value.
#  If the key doesn't match, it pushes the value onto the stack to explore it further.
###

###
#  If current_item is a list, the function iterates over its elements.
#  For each element, it pushes the value onto the stack to explore it further.
###

###
#  This function yields the matching values one at a time as they are found, 
#  rather than finding all matches at once and returning them as a list.
#  This can be useful when working with large nested data structures, as
#   it avoids the need to load the entire structure into memory at once.
###

In [3]:
import pandas as pd
import os
import sys
import time
import json
from IPython.display import display    #used for displaying the comments DataFrame in a Jupyter Notebook or IPython environment.

# Constants
COMMENT_LIMIT = None  # Set the limit for the number of comments to download
FILE_NAME = '廣東話.csv'  # Specify the file name for the CSV file

def main(url):
    """
    This function will save the comments into a DataFrame and output a CSV file
    By default, it will append the comments in the CSV file, not overwrite them.
    To preview the comments in JSON, uncomment the lines - 22 and 23.
    """
    df_comment = pd.DataFrame()
    #  a new DataFrame df_comment is created to store the comments.
    try:
        youtube_url = url
        limit = COMMENT_LIMIT

        print('Downloading YouTube comments for video:', youtube_url)

        count = 0

        start_time = time.time()

        for comment in download_comments(youtube_url):

            df_comment = pd.concat([df_comment, pd.DataFrame([comment])], ignore_index=True)
            # comments overview in JSON
            # comment_json = json.dumps(comment, ensure_ascii=False)
            # print(comment_json)

            count += 1

            if limit and count >= limit:
                break

        print("DataFrame Shape:", df_comment.shape)
        display(df_comment)

        if not os.path.isfile(FILE_NAME):
            df_comment.to_csv(FILE_NAME, encoding='utf-8', index=False)
        else:  # else it exists, append without writing the header
            df_comment.to_csv(FILE_NAME, mode='a', encoding='utf-8', index=False, header=False)

        print('\n[{:.2f} seconds] Done!'.format(time.time() - start_time))

    except Exception as e:
        print('Error:', str(e))
        sys.exit(1)

In [4]:
# dumping youtube comments

""" 
Dump comments to a csv  from a single video

"""
#youtube_URL = 'https://www.youtube.com/watch?v=cD98JOacoAY'
#main(youtube_URL)



import csv

# the function add the encoding parameter "utf-8-sig"
# The "utf-8-sig" encoding includes a UTF-8


# 'w' will create a new csv il
def save_comments_to_csv(comments, filename):
    with open(filename, 'w', encoding='utf-8-sig', newline='') as csvfile:
        fieldnames = ['Video_title','Comment_id', 'Comment_content', 'Time',
                      'Username', 'Channel_user', 'Likes', 'Photo_thumbnails', 'Hearted_by_video_publisher']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for comment in comments:
            #####
            
            
            
            #####
            writer.writerow(comment)

# Call the download_comments function and save the results to a CSV file
YOUTUBE_VIDEO_URL = 'https://www.youtube.com/watch?v=cD98JOacoAY'
main(YOUTUBE_VIDEO_URL)
comments = download_comments(YOUTUBE_VIDEO_URL)

# Specify the filename for the CSV file
filename = '廣東話.csv'

# Save the comments to the CSV file
save_comments_to_csv(comments, filename)





DataFrame Shape: (20, 9)


,Video_title,Comment_id,Comment_content,Time,Username,Channel_user,Likes,Photo_thumbnails,Hearted_by_video_publisher
0,香港揀大學定學科重要啲？DSE失手點做好？成績比預期好應該轉神科嗎？DSE放榜過來人經驗分享...,UgzhYMnNFfGDB6WRzt94AaABAg,三年前開始睇以為好耐先到我原來聽日就到我放榜😢😢，好緊張好驚😭,4 個月前,@HappyHappylife123,UC1etSwj-eQdYQEhK15AXBUA,146,https://yt3.ggpht.com/bHEkenS6J9kkFdMxrC7o5Wrv...,True
1,香港揀大學定學科重要啲？DSE失手點做好？成績比預期好應該轉神科嗎？DSE放榜過來人經驗分享...,UgwfmCxc8KHP08V44HB4AaABAg,今集每個成員嘅分享都好好:yougotthis:\n從唔同嘅角度去俾DSE放榜同學仔意見:v...,4 個月前,@John_Ying,UCcALOQgaKS-Xj6UmQRRUNkQ,5,https://yt3.ggpht.com/ytc/APkrFKYQ-QADrZsGX7we...,True
2,香港揀大學定學科重要啲？DSE失手點做好？成績比預期好應該轉神科嗎？DSE放榜過來人經驗分享...,UgxsptY4OrAT91kaAM14AaABAg,我依家嘅situation就好兩難 best5得16，34333\n我好想讀poly hot...,4 個月前,@jophyyy-tamm._.6025,UC84qSmc8U3bL_xjHLjcn6yA,8,https://yt3.ggpht.com/ytc/APkrFKYtw4b0wHMp4GV2...,False
3,香港揀大學定學科重要啲？DSE失手點做好？成績比預期好應該轉神科嗎？DSE放榜過來人經驗分享...,Ugy4wOy9mQcX_sUDgcZ4AaABAg,作為29歲過來人如果無話特別興趣，家底又一般，絕對係簡專科，千奇唔好因為為大學名簡非專科，除...,4 個月前,@sangson263,UC2rlXqed_tt_gaesmcQ7frQ,52,https://yt3.ggpht.com/ytc/APkrFKYH9WePiLoPai53...,True
4,香港揀大學定學科重要啲？DSE失手點做好？成績比預期好應該轉神科嗎？DSE放榜過來人經驗分享...,UgyZAH1PZGQ7y_C07ux4AaABAg,謝謝你們年輕人👍🏻,3 個月前,@waiyiwongwong8204,UCGp5-9_ZNyLs2m-eVd15p3w,2,https://yt3.ggpht.com/ytc/APkrFKbEUPprDfU0fQWz...,True
5,香港揀大學定學科重要啲？DSE失手點做好？成績比預期好應該轉神科嗎？DSE放榜過來人經驗分享...,Ugxg-27U3VKpqO9LbeN4AaABAg,"屋企人都係做工程項目，做過三間，其中兩間老細都係請Poly唔要Kong u, 因為kong ...",4 個月前,@dorothyc9571,UC3wU0meobOCYg68_xF11xVw,26,https://yt3.ggpht.com/8oqkGqP2Xlfr_pTmXzGP7cd_...,True
6,香港揀大學定學科重要啲？DSE失手點做好？成績比預期好應該轉神科嗎？DSE放榜過來人經驗分享...,UgxpEA6PEIyR9MWScUt4AaABAg,如果dse中英冇3，想靠asso博上大學都係要retake補返跛咗嗰一科，冇得避,4 個月前,@azel6257,UCjsiTAmN9CJ6ZQa-CaAVhyA,4,https://yt3.ggpht.com/ytc/APkrFKbBZsSbJUET3i4q...,False
7,香港揀大學定學科重要啲？DSE失手點做好？成績比預期好應該轉神科嗎？DSE放榜過來人經驗分享...,UgwVjRGzr8T-iFlzNxR4AaABAg,No retake，下年改左制度，好多大學唔承認通識,4 個月前,@user-tf2pn7ll3s,UC7Mhc7-iWHC2Ui8uz0RcK3A,9,https://yt3.ggpht.com/ytc/APkrFKbVDlMidzQZj_BB...,True
8,香港揀大學定學科重要啲？DSE失手點做好？成績比預期好應該轉神科嗎？DSE放榜過來人經驗分享...,UgyLbR_5H2L2V68BuM54AaABAg,淨係想講 如果想讀社工又入唔到U一定要揀hd rather than asso🫠,4 個月前,@noname-fl3tx,UCvf35e-E2w8-EHBGU9rcaDQ,2,https://yt3.ggpht.com/ytc/APkrFKax1pL5b5r2jbm3...,False
9,香港揀大學定學科重要啲？DSE失手點做好？成績比預期好應該轉神科嗎？DSE放榜過來人經驗分享...,UgwR5fJ3GaTSfu9JVuZ4AaABAg,如果系中文麻麻地的英籍国际学生，有机会入港大和中大，边一个校园生活会多元化D？ 边一个会系大...,3 星期前 (已編輯),@SW-fy8pq,UCiTXq6gkV-NLHhCz3inVdlw,0,https://yt3.ggpht.com/ytc/APkrFKYc15Q1dBqZ7jcc...,False



[1.00 seconds] Done!


In [5]:
df_comment = pd.read_csv('廣東話.csv')
df_comment.shape

(20, 9)

In [6]:
ytb_video_list = ['https://www.youtube.com/watch?v=DsaKNGLyWiQ',
                  'https://www.youtube.com/watch?v=XMUjj5-nRF8',
                  'https://www.youtube.com/watch?v=UFPVoDDcU8I&t=1763s',
                  'https://www.youtube.com/watch?v=LPJc5P-ogPo',
                  'https://www.youtube.com/watch?v=aLPfRPAvlpk',
                  'https://www.youtube.com/watch?v=rXNGrNypJc0',
                  'https://www.youtube.com/watch?v=qGqSTEpP2zE',
                  'https://www.youtube.com/watch?v=lFDIqHuctJ0',
                  'https://www.youtube.com/watch?v=IS_BfAEknwQ',
                  'https://www.youtube.com/watch?v=iGqP81fmsyg',
                  'https://www.youtube.com/watch?v=u8MkDjLWxeo'
                  ]

for video_link in ytb_video_list:
    main(video_link)



    
import csv


"""
To avoid overwriting the existing comments in the CSV file and append new comments,
can modify the save_comments_to_csv function to open the file in append mode ('a') instead of write mode ('w').
"""


def save_comments_to_csv(comments, filename):
    with open(filename, 'a', encoding='utf-8-sig', newline='') as csvfile:
        fieldnames = ['Video_title','Comment_id', 'Comment_content', 'Time',
                      'Username', 'Channel_user', 'Likes', 'Photo_thumbnails', 'Hearted_by_video_publisher']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        #writer.writeheader()
        for comment in comments:
            writer.writerow(comment)

"""
# List of YouTube video URLs
YOUTUBE_VIDEO_URLS = ['https://www.youtube.com/watch?v=DsaKNGLyWiQ',
                  'https://www.youtube.com/watch?v=XMUjj5-nRF8',
                  'https://www.youtube.com/watch?v=UFPVoDDcU8I&t=1763s',
                  'https://www.youtube.com/watch?v=LPJc5P-ogPo',
                  'https://www.youtube.com/watch?v=aLPfRPAvlpk',
                  'https://www.youtube.com/watch?v=rXNGrNypJc0',
                  'https://www.youtube.com/watch?v=qGqSTEpP2zE',
                  'https://www.youtube.com/watch?v=lFDIqHuctJ0',
                  'https://www.youtube.com/watch?v=IS_BfAEknwQ',
                  'https://www.youtube.com/watch?v=iGqP81fmsyg',
                  'https://www.youtube.com/watch?v=u8MkDjLWxeo'
                  ]
"""
# Specify the filename for the CSV file
filename = '廣東話.csv'

all_comments = ytb_video_list

# Save the combined comments to the CSV file
save_comments_to_csv(all_comments, filename)

DataFrame Shape: (20, 9)


,Video_title,Comment_id,Comment_content,Time,Username,Channel_user,Likes,Photo_thumbnails,Hearted_by_video_publisher
0,探索最強康文署健身室 I 絕無僅有 - YouTube,Ugx_x8131qTjKXkBky94AaABAg,胡力威 師傅 去咗大埔去康文署做經理🤣🤣,2 個月前 (已編輯),@LeoTang1997,UCxqhelv8ZxvPL2m0OkXpGnw,126,https://yt3.ggpht.com/ytc/APkrFKZnO-biZCq25Prx...,False
1,探索最強康文署健身室 I 絕無僅有 - YouTube,UgzjOFuNNXJlV3PnPrl4AaABAg,多謝你哋🤗辛苦晒💪🏻😘特別係為咗今集，竟然不惜重本去做多6張月票😱真係冇嘢好講🙇🏻‍♂️🫡,2 個月前,@willson_man,UC-8Anq6Aw3VBEpAtlbvsk6A,7,https://yt3.ggpht.com/B635W0r6FEu9njTuhA0jJRdu...,True
2,探索最強康文署健身室 I 絕無僅有 - YouTube,UgwRsttOZjD755OQYrZ4AaABAg,大埔體育館 換入史密夫架之前. 係用BENCH + RACK(無SAFETY SUPPORT...,2 個月前,@wongwilliam5427,UC24UqI-sphoAPMDvsLyD_GQ,9,https://yt3.ggpht.com/ytc/APkrFKZfBEUWhfdHG5io...,False
3,探索最強康文署健身室 I 絕無僅有 - YouTube,Ugw8rnRZHRXBUV7zhgh4AaABAg,試Smith比阿伯問有冇玩開😂😂比阿伯笑細隻,2 個月前,@kl4224,UCgypV7ImAHZLznNmBBB3lyg,58,https://yt3.ggpht.com/_Qtl83yd860371nct0cbJ3mh...,False
4,探索最強康文署健身室 I 絕無僅有 - YouTube,UgwHkplR8SGTXw1xaql4AaABAg,下次大西北一定要去保榮路體育館，極大但極之浪費位置，如果可以用得盡擺位，一定揮低所有康文署G...,2 個月前,@Nick1977Lau,UC0Prmh1vd8kKbalMV9mcL0g,33,https://yt3.ggpht.com/ytc/APkrFKY4cSUDQIlyacwR...,False
5,探索最強康文署健身室 I 絕無僅有 - YouTube,UgwkljP5xBV_WZdJj0d4AaABAg,港灣道基本上過12點就唔駛去，因為12-2 附近office 人襯lunch time 操，...,2 個月前,@siyan200,UCSRl8RhuAVZjms7X_tBRlHw,11,https://yt3.ggpht.com/HQtTa15ORjLdI9dZF0W-bsaM...,False
6,探索最強康文署健身室 I 絕無僅有 - YouTube,UgwAjz4e8M5GuvAA8E54AaABAg,大埔幾間康文署都算足料，但放假同夜晚都係好full同要排隊等位入 ，龍門架/史密夫輪街症咁😂,2 個月前,@wcw1343,UCDja4RW3gjT2hWP_0na71QQ,3,https://yt3.ggpht.com/ytc/APkrFKa8YzPAjB2kXiAt...,False
7,探索最強康文署健身室 I 絕無僅有 - YouTube,UgwL-IshqQst5h5yiet4AaABAg,拍片唔易\n要抽幾日拍\n用心制作\n支持👍\n辛苦晒,2 個月前,@simonchan6323,UC-KbLzMl8gJjutaJU4fxXZA,0,https://yt3.ggpht.com/nBhMKJxA5I044lkaWcO80Aoh...,False
8,探索最強康文署健身室 I 絕無僅有 - YouTube,Ugzb09JDD9VYh4bzKaZ4AaABAg,鍾意呢個系列，特別係介紹器械個part,2 個月前,@DingusSquatfordJr.,UCoDaTgClCjN9ofiEb9lNvfw,3,https://yt3.ggpht.com/ytc/APkrFKYzRimce0GVcHh2...,False
9,探索最強康文署健身室 I 絕無僅有 - YouTube,Ugw1EjnvIHfJ5qD5evd4AaABAg,大埔體育館陪伴左我健身生涯頭幾年 當年仲有大龍門架同埋cybex 機 絕對係最強康文gym ...,2 個月前,@tonywu5c33,UCvJiHUJxYVCZDQ90LOOCAgw,0,https://yt3.ggpht.com/ytc/APkrFKZTHCdAdj9ic9NW...,True



[0.95 seconds] Done!
DataFrame Shape: (20, 9)


,Video_title,Comment_id,Comment_content,Time,Username,Channel_user,Likes,Photo_thumbnails,Hearted_by_video_publisher
0,全港”最細/西”康文署健身室!?|百無!!迫到無朋友!! - YouTube,UgzmNdInL-PvCfu0WYp4AaABAg,不如整個酒店gym room series😂,4 個月前,@binnyl6492,UCAQTnb9CyOMbfaCsdukd4hg,165,https://yt3.ggpht.com/ytc/APkrFKaNvy-7njLynpOx...,True
1,全港”最細/西”康文署健身室!?|百無!!迫到無朋友!! - YouTube,UgwCPX_e7nlXI1QjNcB4AaABAg,其實啞鈴有75磅就算少器材都唔算好差，一對啞鈴都玩到好多動作！,4 個月前,@singpo3896,UCq7P9wkyVPQpH1zGuX6FHww,44,https://yt3.ggpht.com/ytc/APkrFKZvoOzpXE0Mu7Yf...,True
2,全港”最細/西”康文署健身室!?|百無!!迫到無朋友!! - YouTube,Ugz_kT11za5LcfNJ3wd4AaABAg,寶林 / 將軍澳 應該都算細掛,4 個月前,@wahung0619,UC_uLNjuTRsz3ylN4MDgsg0w,18,https://yt3.ggpht.com/ytc/APkrFKbcx1iS1rkLWdhb...,True
3,全港”最細/西”康文署健身室!?|百無!!迫到無朋友!! - YouTube,UgzUzjwueAfpbXnaNRN4AaABAg,你啲片幾好，好寫實，keep住！！！,4 個月前,@wmc5742,UCxGakdrzuZTD2u5Msy5awRQ,12,https://yt3.ggpht.com/ytc/APkrFKZXQGLzp0i-fV6R...,True
4,全港”最細/西”康文署健身室!?|百無!!迫到無朋友!! - YouTube,UgzNcF5YrhJnOeWPpfh4AaABAg,不如拍晒全香港所有康文署健身室系列。全制霸😂🎉正,4 個月前,@moonkk2028,UCK7SZ94-3jYRSQHXzN-lcuw,134,https://yt3.ggpht.com/ytc/APkrFKayTkUkMsOK_hr_...,True
5,全港”最細/西”康文署健身室!?|百無!!迫到無朋友!! - YouTube,Ugy--MT7GUgHT-R1ogl4AaABAg,"曉光街同順利大概距離只差1公里左右, 設施好好多",4 個月前,@user-mc3ue1ot4o,UCxUfgS1hJBQHcQA9ZNaYhjw,9,https://yt3.ggpht.com/ytc/APkrFKYZbXvsg9lF-50F...,True
6,全港”最細/西”康文署健身室!?|百無!!迫到無朋友!! - YouTube,UgwZIbDwkTwQde7NgHp4AaABAg,鳯琴街先係細場地獄😂,4 個月前,@chanchris5190,UCMQ-3JF_lT_VJnTQuE8YZyg,27,https://yt3.ggpht.com/ytc/APkrFKbIwco74FpfY2fv...,True
7,全港”最細/西”康文署健身室!?|百無!!迫到無朋友!! - YouTube,UgwHvblYDXEoXZmXypN4AaABAg,仲諗住有恒安邨體育館😂,4 個月前,@TheColonelbruce,UCpNfsxmq3-VfWku0trwlNAQ,25,https://yt3.ggpht.com/ytc/APkrFKZH28VcRClVNO6U...,True
8,全港”最細/西”康文署健身室!?|百無!!迫到無朋友!! - YouTube,UgyhYRykYuXkMwAEx4l4AaABAg,上水天平係全香港最細最西康文gym room 連啞鈴都冇😂😂😂😂,4 個月前,@cronaldo4972,UC9UtwYkTP44J683rUGtX1qw,68,https://yt3.ggpht.com/ytc/APkrFKboDJYaVAliwjrB...,True
9,全港”最細/西”康文署健身室!?|百無!!迫到無朋友!! - YouTube,UgwXs9Aq5bce6FZTY_Z4AaABAg,梨木樹鄧肇堅體育館真係大魔王 細到仆街😵‍💫🫠,4 個月前,@heihei7952,UCSU3bXneX_LwXEQYCm8PQTQ,31,https://yt3.ggpht.com/ytc/APkrFKbBSnxxWKj-wGfY...,True



[0.92 seconds] Done!
DataFrame Shape: (20, 9)


...

,Video_title,Comment_id,Comment_content,Time,Username,Channel_user,Likes,Photo_thumbnails,Hearted_by_video_publisher
0,Karl首戰 l 轟轟烈烈 l 有前無後打死罷就 - YouTube,UgwhgCUwur8nG-rm3Ix4AaABAg,Bulk得咁差都仲比佢衝，其實Terry作為教練都好大責任，唔係一味熱血想做就係正確,3 個月前,@chukk4488,UCHBu0EmEvfsLv8YyNw2HR6g,74,https://yt3.ggpht.com/ytc/APkrFKa31jj7C0R7EUDV...,False
1,Karl首戰 l 轟轟烈烈 l 有前無後打死罷就 - YouTube,UgyvLl0zCgzncpXPIQN4AaABAg,真係好有heart！有決心和堅持，這點毫無疑問！！！\n但估不到肌肉量這麼弱，很驚訝！因一直...,3 個月前 (已編輯),@dennischow9924,UCQLBZ65liHO5q3REySAv34w,46,https://yt3.ggpht.com/ytc/APkrFKbLvYM0P8OCnOcf...,True
2,Karl首戰 l 轟轟烈烈 l 有前無後打死罷就 - YouTube,UgyZzQHIrNuP139VrwJ4AaABAg,以前比賽前修身踩機，d汗係咁滴入眼，又帶眼鏡，都唔知係汗定流緊眼淚😢真係好辛苦，但賽後真係知...,3 個月前,@5eric6man7,UCOi_I2ssErnx8m1xzAKxMMw,25,https://yt3.ggpht.com/ytc/APkrFKaWf0yCiYtEG6Bw...,True
3,Karl首戰 l 轟轟烈烈 l 有前無後打死罷就 - YouTube,UgymYRRXSG9dxS0nSdh4AaABAg,真心睇得出你好比心機 繼續努力 呢次係你踏向成功既第一步❤,3 個月前,@wakandaforever958,UCpTSS6PL2-JT1kYeWtvmZLQ,13,https://yt3.ggpht.com/17we64mKMXi2fH6IrMyl921B...,True
4,Karl首戰 l 轟轟烈烈 l 有前無後打死罷就 - YouTube,Ugw8DvxmzF__yvnq0kl4AaABAg,能夠捱到上到台，已經達標!\n成績唔緊要，下次努力上磅，再修身，必定更好👍👍👍👍,3 個月前,@bigshing2002,UCcjY4ByJWGjc_di2qMHJ8fg,7,https://yt3.ggpht.com/ytc/APkrFKZcFfhHCCJAtaYz...,True
5,Karl首戰 l 轟轟烈烈 l 有前無後打死罷就 - YouTube,Ugx4JrduC39KPo8ed1J4AaABAg,真心尊重🫡 過程好靠意志力，唔易做到🫡,3 個月前,@CathyTsang,UCJOVVBJwuWi3JvZFFlkhuiQ,6,https://yt3.ggpht.com/ytc/APkrFKYMdqagL37Ifcyt...,True
6,Karl首戰 l 轟轟烈烈 l 有前無後打死罷就 - YouTube,UgywNbUTQVANOUs_9xZ4AaABAg,KOFGYM最感動的一集👍,3 個月前,@briankwong456,UCgfxCpCA8reH9eMxFvUfYKg,14,https://yt3.ggpht.com/ytc/APkrFKZfzzcz_Kk1HBJ_...,True
7,Karl首戰 l 轟轟烈烈 l 有前無後打死罷就 - YouTube,Ugxw7aGMMavBI41dq3B4AaABAg,"🍔:我再睇反你上年參賽時的視頻,唔知點解哭了出嚟,多謝你哋的認真、勇氣,勉勵咗我😸👍👍👍",3 個月前,@abbylo1216,UCWyPoKi-k-GJjSkdosKUW9A,9,https://yt3.ggpht.com/ytc/APkrFKYgBtjhSriI79Ux...,True
8,Karl首戰 l 轟轟烈烈 l 有前無後打死罷就 - YouTube,UgzJVLBAIPl7M2lUzrZ4AaABAg,"你有這個目標,已經贏咗自己,享受這個出場表演機會,唔係唾手可得👍👍👍",3 個月前,@abbylo1216,UCWyPoKi-k-GJjSkdosKUW9A,3,https://yt3.ggpht.com/ytc/APkrFKYgBtjhSriI79Ux...,True
9,Karl首戰 l 轟轟烈烈 l 有前無後打死罷就 - YouTube,Ugw61b1Mne777_ZRKPt4AaABAg,好睇 好熱血 果句健美比賽係咁 聽落去好不甘 好有感覺 加油。,3 個月前,@kwanchris7762,UClvgSf0X903YwiIoRiiDibg,5,https://yt3.ggpht.com/ytc/APkrFKbTlSwaSg_mwgpw...,True



[0.85 seconds] Done!
DataFrame Shape: (20, 9)


,Video_title,Comment_id,Comment_content,Time,Username,Channel_user,Likes,Photo_thumbnails,Hearted_by_video_publisher
0,大爆料🔥 ChillUp吵架實錄... 這半年間不想提及的一件事💬人際關係真的好複雜😂😂 友...,UgziFAOhRrtK8EWInvV4AaABAg,▼ 試用 Surfshark VPN: \nhttps://surfshark.deals/...,5 個月前,@a.antoniaw,UCi6CqLjdoCN_ijofoCJFpCw,2,https://yt3.ggpht.com/FsC9Acy9VvUw_25Abe8rzaQE...,False
1,大爆料🔥 ChillUp吵架實錄... 這半年間不想提及的一件事💬人際關係真的好複雜😂😂 友...,UgzFoAwMo_xYvcTMMmh4AaABAg,果然真。八卦特別好睇🤣笑死,5 個月前,@Killer1125,UC7jRMz1jKWHLGJ1upKyYo-g,20,https://yt3.ggpht.com/uasLwfQJgtWlXIu8xwEYGYPY...,True
2,大爆料🔥 ChillUp吵架實錄... 這半年間不想提及的一件事💬人際關係真的好複雜😂😂 友...,Ugyh49DvotGuo5FczUt4AaABAg,今集好搞笑得嚟又學到啲人際關係上嘅嘢☀️好睇!\nAnton同Tim嘅火花應該係Chill ...,5 個月前,@John_Ying,UCcALOQgaKS-Xj6UmQRRUNkQ,12,https://yt3.ggpht.com/ytc/APkrFKYQ-QADrZsGX7we...,True
3,大爆料🔥 ChillUp吵架實錄... 這半年間不想提及的一件事💬人際關係真的好複雜😂😂 友...,Ugzd0CfVUUd4WZOKX7N4AaABAg,係敏感嘅Timsung遇上無咁敏感嘅其他人同極唔敏感嘅Anton\n件衫真係有啲癲😂\n原來...,5 個月前,@hongkonger1111,UCWM1pduxDAvDadF3pc35lCQ,9,https://yt3.ggpht.com/ytc/APkrFKYYtxZBokQtHi46...,True
4,大爆料🔥 ChillUp吵架實錄... 這半年間不想提及的一件事💬人際關係真的好複雜😂😂 友...,UgzSNLG9f3_vh5kr0JV4AaABAg,"好同意Tim個講法\n就算對方講左sorry, sorry完之後又繼續重覆做同一件事\n呢個...",5 個月前,@karyiufong3659,UCwjHyJbOT15a4wmLQWjnB2Q,9,https://yt3.ggpht.com/ytc/APkrFKap0mw_nrBSafqL...,True
5,大爆料🔥 ChillUp吵架實錄... 這半年間不想提及的一件事💬人際關係真的好複雜😂😂 友...,Ugy4Nu_ruX-_u0LEZJ94AaABAg,8:30好少見Violet嬲到叫人收皮😅😅證明今次真係好嬲😶‍🌫😶‍🌫希望可以拍多D呢一類型...,5 個月前,@gabrielle9829,UCE-T5dz_QtFDXGD8YVZk8zQ,9,https://yt3.ggpht.com/ytc/APkrFKZrQo35_qM06f9a...,False
6,大爆料🔥 ChillUp吵架實錄... 這半年間不想提及的一件事💬人際關係真的好複雜😂😂 友...,UgwQbHYSgBGAYgHJQS14AaABAg,我明Tim點解嬲喎😂😂😂 俾我開個工作group然後大家夾硬要喺入面討論啲得少少關係嘅嘢，我...,5 個月前,@joeylam.827,UCb1SwJq92Sj62Ru0f_aDyKg,14,https://yt3.ggpht.com/ytc/APkrFKa5RSUWa9YA5F2K...,False
7,大爆料🔥 ChillUp吵架實錄... 這半年間不想提及的一件事💬人際關係真的好複雜😂😂 友...,UgxDW1p8I5nmFxe-z8N4AaABAg,幾經辛苦終於出到哩條片😂😂,5 個月前,@Zoeysy,UCuJH_j_pvS56qkZtxyvnW4w,9,https://yt3.ggpht.com/ZFgAvPedw3MyeX3sfdCxdt0L...,True
8,大爆料🔥 ChillUp吵架實錄... 這半年間不想提及的一件事💬人際關係真的好複雜😂😂 友...,Ugy45Qb97iNkup8tYIt4AaABAg,😂😂😂😂😂唔講都唔記得點解timtim會嬲，默默地係gp食左好耐花生😂,5 個月前,@violetssss_,UCQaP6KYdVbciSMGhIUoivow,5,https://yt3.ggpht.com/dOziDl5sc30iOAXi3qCSc0eF...,True
9,大爆料🔥 ChillUp吵架實錄... 這半年間不想提及的一件事💬人際關係真的好複雜😂😂 友...,UgzgWuYHHgJtUv37Y0Z4AaABAg,開頭三五分鐘完全係笑住笑住咁睇過來嘅😂😂😂😂😂😂,5 個月前,@chungjai20,UCyWg1J9LSa0DY7drhOfGxqA,11,https://yt3.ggpht.com/HdXa2-XHOPFOLzxXfbtbNqYf...,True



[0.83 seconds] Done!


AttributeError: 'str' object has no attribute 'keys'

In [7]:
df_comment = pd.read_csv('廣東話.csv')
df_comment.shape

(240, 9)